In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
# with_info = True Prodives a tuple containing info about the version
# as_supervised = True loads the data in a two tuple structure
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised= True)

# Data

In [3]:
mnist_train ,mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples #10 % of trianing
num_validation_samples = tf.cast(num_validation_samples , tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255. # . means the result should be float
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE =10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE =100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# iter = creates an object which can be iterated, next = loads the next element of an iterable object 
validation_inputs, validation_targets = next(iter(validation_data))

# Model

In [4]:
input_size = 784 # 28*28 as an image is of 28 * 28 pixel matrix
output_size = 10 # one for each number from 0-9
hidden_layer_size = 100 # assumption widht of same size

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    # tf.keras.layers.Dense(output size) 
    # takes the inputs provided to the model and calculates the dot product of the inputs and weights and add the bias.
    # This is also where we can apply activation function
    tf.keras.layers.Dense(hidden_layer_size, activation ='relu'), # Hidden Layer
    tf.keras.layers.Dense(hidden_layer_size, activation ='relu'),  # Hidden Layer
    tf.keras.layers.Dense(output_size, activation ='softmax')     # Output Layer
    
])

# Optimizer and Loss Function

In [5]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Training

In [6]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)


Epoch 1/5
540/540 - 8s - loss: 0.3370 - accuracy: 0.9034 - val_loss: 0.1601 - val_accuracy: 0.9542
Epoch 2/5
540/540 - 4s - loss: 0.1363 - accuracy: 0.9602 - val_loss: 0.1121 - val_accuracy: 0.9647
Epoch 3/5
540/540 - 5s - loss: 0.0970 - accuracy: 0.9705 - val_loss: 0.0903 - val_accuracy: 0.9722
Epoch 4/5
540/540 - 5s - loss: 0.0744 - accuracy: 0.9777 - val_loss: 0.0784 - val_accuracy: 0.9753
Epoch 5/5
540/540 - 5s - loss: 0.0613 - accuracy: 0.9807 - val_loss: 0.0834 - val_accuracy: 0.9740


# Test

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 1ms/step - loss: 0.1087 - accuracy: 0.9668


In [8]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100.))

Test loss: 0.11. Test accuracy: 96.68%
